challenge 4 (crimen en la libreria SQL):    

In [5]:
# ------- almacen de datos ------
authors_books = []

In [6]:
# ------- funciones importantes -------

import requests
import time


# ----- funcion para hacer request a los url  ------
def hacer_requests(URL, params=None):
    reintentos_posibles = 5
    tiempo_espera = 1
    
    time.sleep(tiempo_espera)
    for intentos in range(reintentos_posibles):
        try:
            respuesta = requests.get(URL, params=params)
            return respuesta
        except:
            print(f"[error de conexion] : [intento {intentos + 1}] : [tiempo de espera : {tiempo_espera}]")
            time.sleep(tiempo_espera)
            tiempo_espera *= 2
    
    return None

In [ ]:
# ------- Escraper en si -------

from bs4 import BeautifulSoup
from urllib import parse

# ----- URL a usar ------
url_booksToScrape = "https://books.toscrape.com/index.html"
url_api_googleBooks = "https://www.googleapis.com/books/v1/volumes"

# ----- hacer conexion y soup ------
respuesta = hacer_requests(url_booksToScrape)
soup = BeautifulSoup(respuesta.content, "lxml")

# ------ recoger todas las categorias -----
side_categories_tag = soup.find("div", class_="side_categories")
nav_list_tag = side_categories_tag.find("ul", class_="nav nav-list")
ubication_categories_tag = nav_list_tag.find("ul")
list_all_categories_tag = ubication_categories_tag.find_all("li")

for elemento in list_all_categories_tag:
    
    # ------- nombre de la categoria --------
    category_name = elemento.a.get_text(strip=True)
    # ----------------------------------------

    print(category_name)

    category_reference_url = elemento.a["href"]

    new_url = parse.urljoin(url_booksToScrape, category_reference_url)
    response = hacer_requests(new_url)
    category_scan = True

    while category_scan:
        category_eleccion_soup = BeautifulSoup(response.content, "lxml")

        # --------- encontrar todos los libros de la categoria --------
        div_tag = category_eleccion_soup.find("div", class_="col-sm-8 col-md-9")
        section_tag = div_tag.find("section")
        ol_tag = section_tag.find("ol", class_="row")
        
        list_of_books_tags = ol_tag.find_all("article", class_="product_pod")

        for book in list_of_books_tags:

            # ------- nombre del libro ----------
            book_name = book.div.a.img["alt"]
            # -----------------------------------

            # proceso para obtener autores
            params = {
                "q" : f"intitle:{book_name}"
            }

            try:
                response_api = hacer_requests(url_api_googleBooks, params)
                data = response_api.json()
                items = data["items"][0]
                volume_info = items["volumeInfo"]

                # --------- autores del libro ----------
                authors = volume_info.get("authors", [])
                # --------------------------------------

                if len(authors) == 0:
                    # por si la lista es vacia
                    authors = ["none"]

            except:
                # por si google me deniega
                authors = ['none']

            # ------ cantidad de estrellas ------
            stars = book.p["class"][1]
            # -----------------------------------

            product_price_tag = book.find("div", class_="product_price")

            # ------ precio del libro ---------------------
            price = product_price_tag.p.get_text(strip=True)
            # ----------------------------------------------

            p_tag = product_price_tag.find("p", class_="instock availability")

            # ------- si el libro esta en stock ------------
            in_stock = p_tag.get_text(strip=True)
            # ----------------------------------------------

            authors_books.append((category_name, {
                "title" : book_name,
                "authors" : authors,
                "stars" : stars,
                "price" : price,
                "stock" : in_stock
            }))
            print(f"[agregado] : [book : {book_name}] : [author : {authors}]")
        
        next_tag = section_tag.find("li", class_="next")
        
        if next_tag:
            next_url = next_tag.a["href"]
            next_page_url = parse.urljoin(new_url, next_url)
            print(f"[next page : {category_name}]")
            response = hacer_requests(next_page_url)
            continue

        break





    


Travel
[agregado] : [book : It's Only the Himalayas] : [author : ['S. Bedford']]
[agregado] : [book : Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond] : [author : ['Rick Antonson']]
[agregado] : [book : See America: A Celebration of Our National Parks & Treasured Sites] : [author : ['none']]
[agregado] : [book : Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel] : [author : ['Rolf Potts']]
[agregado] : [book : Under the Tuscan Sun] : [author : ['Frances Mayes']]
[agregado] : [book : A Summer In Europe] : [author : ['Mary Elizabeth Blake']]
[agregado] : [book : The Great Railway Bazaar] : [author : ['Paul Theroux']]
[agregado] : [book : A Year in Provence (Provence #1)] : [author : ['Peter Mayle']]
[agregado] : [book : The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)] : [author : ['Bill Bryson']]
[agregado] : [book : Neither Here nor There: Travels in Europe] : [author : ['Bill Bryson']]
[agregado] : [bo

In [ ]:
import sqlite3

conexion = sqlite3.connect("dataBase.db")
cursor = conexion.cursor()

cursor.execute("BEGIN")

for category, book in authors_books:

    cursor.execute("""INSERT OR IGNORE INTO Categories(category_name) VALUES (?)""", (category,))

    cursor.execute("SELECT id FROM Categories WHERE category_name = ?", (category,))

    category_id = cursor.fetchone()[0]

    # desempaquetar informacion del libro:
    title = book["title"]
    authors_book = book["authors"]
    stock = book["stock"]


    try:
        stars = ["Zero","One","Two","Three","Four","Five"].index(book["stars"].capitalize())
    except:
        stars = None

    price = float(book["price"].replace("£","").strip()) if "£" in book["price"] else None

    cursor.execute("INSERT OR IGNORE INTO Books(title, price, stars, stock, id_category) VALUES (?, ?, ?, ?, ?)", (title, price, stars, stock, category_id))

    cursor.execute("SELECT id FROM Books WHERE title = ?", (title,))

    book_id = cursor.fetchone()[0]

    for aut in authors_book:

        cursor.execute("INSERT OR IGNORE INTO Authors(name) VALUES (?)", (aut,))

        cursor.execute("SELECT id FROM Authors WHERE name = ?", (aut,))

        author_id = cursor.fetchone()[0]

        cursor.execute("INSERT OR IGNORE INTO Books_authors(id_book, id_author) VALUES (?, ?)", (book_id, author_id))

    print(f"[agregado] : {title}")

conexion.commit()
conexion.close()